I'm going to give a demonstration of some basic techniques and principles for optimising your Python code (i.e. making it run faster), with the example of calculataing prime numbers under 10000. Let's begin with a straightforward way to calculate primes:

As a reminder, a prime number is a natural number ($\mathbb{N}$) that is greater than 1 and is not the product of two smaller natural numbers.

In order to determine whether a number is the product of two smaller natural numbers, we can use the _modulo_ operator `%`, which given $a \% b$, outputs the remainder of $a/b$. Here's an example:

In [1]:
print(3 % 2)
print (5 % 5)
print (10 % 4)

1
0
2


Now to generate our list of primes, I propose the following algorithm:

- INITIALIZATION:
    - Container of prime numbers found thus far. Insert the first prime, $2$.
    - Set flag `isPrime` to `True`
- ITERATION: FOR all integers `i` greater than 2 and less than 10000:
    - ITERATION FOR: each prime number already found `p`
        - CONDITION: IF $i\%p=0$:
            - Set flag `isPrime` to `False`
    - END ITERATION:
        - CONDITION: IF `isPrime` is `True`
            - Append `p` to list of prime numbers
            - Reset flat to initialization
- END ITERATION

In words, I check whether each number is divisible with zero remainder by all the primes less than itself. If it is, then I add it to the list of primes.

One might note that there is a lot of redundancy in this algorithm; we will get to that.

In [2]:
isPrime = True
primes = [2] # An empty list

for i in range(3, 10001): # Start from 1, finish at 10000
    # Check if divisible with no remainder from existing arguments
    for p in primes:
        # Here we need some kind of conditional logic. I will use 'if' again.
        if i % p == 0: # If zero remainder, then multiple and not prime
            isPrime = False
        # After all primes checked, if still isPrime=True, then we can append
    if isPrime == True:
        primes.append(i)
    isPrime = True # Resetting the 'trigger'

Now here's the first trick for optimising code: the `%%timeit` magic available in iPython. This returns the average of how long it takes to run a code block, trying several runs:

In [3]:
%%timeit

isPrime = True
primes = [2] # An empty list

for i in range(3, 10001): # Start from 1, finish at 10000
    # Check if divisible with no remainder from existing arguments
    for p in primes:
        # Here we need some kind of conditional logic. I will use 'if' again.
        if i % p == 0: # If zero remainder, then multiple and not prime
            isPrime = False
        # After all primes checked, if still isPrime=True, then we can append
    if isPrime == True:
        primes.append(i)
    isPrime = True # Resetting the 'trigger'

354 ms ± 57.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


The output of `%%timeit` tells us that the above way of finding all prime numbers less than 10000 takes 281 milliseconds on average, with a standard deviation of 2.23 milliseconds. It also tells us that it ran the code 7 times. (Note that I have am running this on a relatively normal laptop with an i7-8550u (1.8GHz) processor. In fact, the single core speed is relatively slow on this laptop, which is a disadvantage for Python. But I'll get to that.)

## Step 1: Reducing Redundant Calculations

The code we have here is a _nested for-loop_. As you might now, a for-loop is a structure that runs the same operation for each element of an array. A nested for-loop is a for-loop inside another for-loop. So for each element of the outer array, we run another loop over some other array.

In this example, the outer array is all numbers between 3 and 10000, and the inner array is a growing list of prime numbers. Note that this also means that while the time each operation within the inner loop takes should remain more or less constant, the time taken for each iteration of the outer loop will grow because the length of the inner array increases each time a new prime is found.

In much simpler terms: when there is only one prime, each inner loop (checking $i$ against all prime numbers found thus far) is a single loop. As the number of primes grows, the length of time taken to check each of the primes found thus far accordingly grows. So earlier loops take less time than later ones.

We can reduce the total time taken by decreasing the total number of iterations. An obvious first step is to reduce the outer array by excluding all even numbers; we know that they aren't prime, so no need to check.

In [4]:
%%timeit

isPrime = True
primes = [2]

for i in range(3, 10001, 2): # Third argument is step
    for p in primes:
        if i % p == 0:
            isPrime = False
    if isPrime == True:
        primes.append(i)
    isPrime = True

192 ms ± 37.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


That nearly halved the time it took to run, which makes sense; we roughly halved the number of total loops it needed to make.

But the inner loop can be sped up as well. Note that once we find that $i$ is divisible by some $p$, then it is not necessary to check subsequent values of $p$. We can end the inner loop early using a `break` statement.

In [5]:
%%timeit

isPrime = True
primes = [2]

for i in range(3, 10001, 2):
    for p in primes:
        if i % p == 0:
            isPrime = False
            break
    if isPrime == True:
        primes.append(i)
    isPrime = True

41.7 ms ± 2.77 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


We've cut it down again by a factor of 5! (Not five factorial.)

Finally, we can cut down the number of calculations even further by excluding all primes larger than the square root of $i$. I add another conditional and break:

In [6]:
%%timeit

isPrime = True
primes = [2]

for i in range(3, 10001, 2):
    for p in primes:
        if i % p == 0:
            isPrime = False
            break
        if p > (i**0.5): # Exclude all p > sqrt(i)
            break
    if isPrime == True:
        primes.append(i)
    isPrime = True

6.38 ms ± 338 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Without introducing any new libraries or tools, we've sped up our code to be roughly 40 times faster. Moreover, these gains will increase as we try to increase the total number of primes we are trying to find.

## Step 2: Identifying Expensive Calculations

We've eliminated a big chunk of the running time by just removing redundant calculations. The next step is identifying what calculations are taking a lot of time.

Here a knowledge of different libraries and tools is useful. Python provides endless options for numerical arrays, and many boast of their speed and performance.

I will compare four different options for storing integers:

- Lists
- Deques
- `numpy` arrays
- `pandas` Series

Lists are the basic Python mutable array. Deques are another base Python array (from the `collections` library) designed to be faster for appends and other operations on the ends of the array. `numpy` arrays are a standard container for numerical arrays used for numerical computing, and `pandas` is our old-time favourite 1-dimensional data container.

Let's compare the time it takes. (I'll put it into a function to make it a bit more concise).

In [7]:
from collections import deque
import numpy as np
import pandas as pd

def calculate_primes(lower=3, upper=10001, container='list'):
    if container=='list':
        primes = list()
        primes.append(2)
    elif container=='deque':
        primes = deque()
        primes.append(2)
    elif container=='numpy':
        primes = np.array([2])
    elif container=='pandas':
        primes = pd.Series(2)

    isPrime = True
    for i in range(lower, upper, 2):
        for p in primes:
            if i % p == 0:
                isPrime = False
                break
            if p > (i**0.5): # Exclude all p > sqrt(i)
                break
        if isPrime == True:
            if container=='numpy':
                primes = np.append(primes, i)
            elif container=='pandas':
                primes = primes.append(pd.Series(i))
            else:
                primes.append(i)
        isPrime = True
    #return primes

In [8]:
%%timeit
calculate_primes(container='list')

6 ms ± 379 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
%%timeit
calculate_primes(container='list')

6.64 ms ± 306 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
%%timeit
calculate_primes(container='numpy')

62.5 ms ± 367 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
%%timeit
calculate_primes(container='pandas')

376 ms ± 2.93 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


What's going on? `numpy` and `pandas` were much slower than deques or lists!

We can use the %prun magic to break this down. This runs a _profiler_, which tracks all the individual function calls.

In [12]:
%prun calculate_primes(container='list')

         1233 function calls in 0.010 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.010    0.010    0.010    0.010 <ipython-input-7-51b6ef22377d>:5(calculate_primes)
     1229    0.000    0.000    0.000    0.000 {method 'append' of 'list' objects}
        1    0.000    0.000    0.010    0.010 {built-in method builtins.exec}
        1    0.000    0.000    0.010    0.010 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

In [13]:
%prun calculate_primes(container='deque')

         1233 function calls in 0.007 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.007    0.007    0.007    0.007 <ipython-input-7-51b6ef22377d>:5(calculate_primes)
     1229    0.000    0.000    0.000    0.000 {method 'append' of 'collections.deque' objects}
        1    0.000    0.000    0.007    0.007 {built-in method builtins.exec}
        1    0.000    0.000    0.007    0.007 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

We can see that deque and list are pretty similar in terms of the number of function calls. This makes sense, as both are base python objects (and do not rely on underlying objects).

In [14]:
%prun calculate_primes(container='numpy')

         20881 function calls (18425 primitive calls) in 0.084 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.067    0.067    0.084    0.084 <ipython-input-7-51b6ef22377d>:5(calculate_primes)
3684/1228    0.007    0.000    0.016    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
     2457    0.002    0.000    0.002    0.000 {built-in method numpy.array}
     1228    0.002    0.000    0.015    0.000 function_base.py:4690(append)
     1228    0.001    0.000    0.004    0.000 fromnumeric.py:1716(ravel)
     1228    0.001    0.000    0.007    0.000 <__array_function__ internals>:2(concatenate)
     1228    0.001    0.000    0.017    0.000 <__array_function__ internals>:2(append)
     1228    0.001    0.000    0.001    0.000 {method 'ravel' of 'numpy.ndarray' objects}
     2456    0.001    0.000    0.003    0.000 _asarray.py:110(asanyarray)
     1228    0.001    0.000    0.005    0.0

Whereas the previous examples only ran 1233 calls, `numpy`  requires 20881 calls! It looks like resizing a `numpy` array is quite an expensive thing to do (which we should have already known).

In [15]:
%prun calculate_primes(container='pandas')

         1125028 function calls (1118884 primitive calls) in 0.679 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   219923    0.054    0.000    0.085    0.000 {built-in method builtins.isinstance}
     2457    0.023    0.000    0.389    0.000 series.py:209(__init__)
    79831    0.020    0.000    0.030    0.000 generic.py:30(_check)
        1    0.020    0.020    0.679    0.679 <ipython-input-7-51b6ef22377d>:5(calculate_primes)
     4913    0.019    0.000    0.036    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
     1228    0.018    0.000    0.180    0.000 concat.py:306(__init__)
     1229    0.017    0.000    0.027    0.000 {pandas._libs.lib.maybe_convert_objects}
   119130    0.016    0.000    0.016    0.000 {built-in method builtins.getattr}
     2457    0.015    0.000    0.180    0.000 construction.py:423(sanitize_array)
    17192    0.013    0.000    0.036    0.000 common.py:1470(is_e

This is a little bit ridiculous. It's clear why it takes fifty times as long to use pandas as base Python in this instance; there are over a million function calls

## Takeaway

The takeaway is not that you should not use `numpy` or `pandas`! Quite the contrary, both of these libraries provide data containers with an enormous number of convenient built-in functionality. Moreover, neither of these containers are optimised with appending in mind; they are very fast for vectorised matrix operations. If we had a different task, they would likely outperform nested lists or deques.

Deques are probably the best base Python object for this kind of task (there may be others I don't know about). But most of our savings came from writing simply more efficient code.

Obviously, writing efficient code takes time and effort. You need to think through all of the little puzzles of your task, and this is a distraction when you are simply trying to prototype some task. At the end of the day, as a relatively new coder, you will spend far more time writing your code than waiting for it to finish executing. This is part of the beauty of Python! It's very easy to prototype code.

My advice is to keep in mind whether steps you are taking are redundant, and to determine whether avoiding such redundancy is simple. If it will only take 5 minutes to implement some logic to avoid millions of pointless operations, then do it! Otherwise consider carefully whether learning `C++` is a good use of your time.